Full code to make all waypoints from road sections 

In [10]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
import glob
import os
try:
    sys.path.append(glob.glob('./carla/dist/carla-0.9.14-py3.7-win-amd64.egg')[0])
except IndexError:
    print('Couldn\'t import Carla egg properly')
import carla

try:
    sys.path.insert(0,r'C:\Users\ashokkumar\source\repos\AD\carla')
except IndexError:
    pass

from carla import ColorConverter as cc
from agents.navigation.behavior_agent import BehaviorAgent   # type: ignore
from agents.navigation.basic_agent import BasicAgent  # type: ignore

SECONDS_PER_EPISODE = 25

SECONDS_PER_EPISODE = 10  # seconds per episode maybe needs to be removed later
FIXED_DELTA_SECONDS = 0.02 # seconds per frame
SHOW_PREVIEW = True # Show preview for debugging
NO_RENDERING = False # No rendering for training
SYNCRONOUST_MODE = False # Synchronous mode for training
SPIN = 10   # angle of spin for initail spawn
HEIGHT = 480 # height of image
WIDTH = 640 # width of image
SHOW_CAM = SHOW_PREVIEW     # render camera
front_camera = None
CAMERA_POS_Z = 1.3      # camera position similar to tesla model 3
CAMERA_POS_X = 1.4    # camera position similar to tesla model 3
im_width = WIDTH
im_height = HEIGHT


In [11]:
# connect to the sim 
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
world = client.load_world('Town02')

settings = world.get_settings()
settings.no_rendering_mode = NO_RENDERING
settings.synchronous_mode = SYNCRONOUST_MODE
settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
world.apply_settings(settings)
blueprint_library = world.get_blueprint_library()
model_3 = blueprint_library.filter("vehicle.tesla.model3")


# get map look at the map
if SHOW_CAM:
    spectator = world.get_spectator()

town_map = world.get_map()
roads = town_map.get_topology()



: 

In [3]:

# set up route generations
all_waypoints = town_map.generate_waypoints(1) #note distance between waypoints in meters

# make unique
waypoints = []
for wp in all_waypoints:
    if len(waypoints) == 0:
        waypoints.append(wp) #first waypoint is added regardless to start the list
    else:
        found = False
        for uwp in waypoints: #check for same located waypoints and ignore if found
            if abs(uwp.transform.location.x - wp.transform.location.x) < 0.1 \
                            and abs(uwp.transform.location.y - wp.transform.location.y)<0.1 \
                            and abs(uwp.transform.rotation.yaw - wp.transform.rotation.yaw)<20:  #this checks same direction
                found = True
                break
        if not found:
            waypoints.append(wp)


# draw all point in the sim for 60 seconds
for wp in waypoints:
    world.debug.draw_string(wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)

#move spectator for top down view to see all points 
# spectator_transform = random.choice(town_map.get_spawn_points())
# spectator_transform.Rotation += carla.Rotation(pitch=-90, yaw=0, roll=0)
# spectator.set_transform(spectator_transform)

In [4]:
'''
now we will see how this will be used in real examples, like during RL training
e.g. see how long it takes to find closest point to the car

'''
#remove any cars/clean up the sim
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

blueprint_library = world.get_blueprint_library()
# Now let's filter all the blueprints of type 'vehicle' and choose one
bp = blueprint_library.find('vehicle.tesla.cybertruck')
transform = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(bp, transform)  
spectator = world.get_spectator()
spectator_transform =  vehicle.get_transform()
spectator_transform.location += carla.Location(z = 20.0)
spectator_transform.rotation.pitch = -90
world.get_spectator().set_transform(spectator_transform)


agent = BasicAgent(vehicle)
waypoint_counter = 0


In [5]:
print(vehicle.get_location())


Location(x=189.929993, y=121.209999, z=0.215703)


In [6]:
initial_pos = vehicle.get_location()
curr_waypoint = initial_pos
curr_distance = 1000
for wp in waypoints:
    dist = curr_waypoint.distance(wp.transform.location)
    if dist < curr_distance:
        curr_distance =  dist
        selected_wp = wp
print(initial_pos)
print(selected_wp.transform.location)


Location(x=189.929993, y=121.209999, z=0.215703)
Location(x=189.735886, y=121.087326, z=0.000000)


In [7]:
next_wp = selected_wp.next(20.0)[0]
print(next_wp)


Waypoint(Transform(Location(x=189.708725, y=141.087311, z=0.000000), Rotation(pitch=360.000000, yaw=90.077835, roll=0.000000)))


In [8]:
print(selected_wp.transform)
print(next_wp.transform)

Transform(Location(x=189.735886, y=121.087326, z=0.000000), Rotation(pitch=360.000000, yaw=90.077835, roll=0.000000))
Transform(Location(x=189.708725, y=141.087311, z=0.000000), Rotation(pitch=360.000000, yaw=90.077835, roll=0.000000))


In [9]:
veh_trans = vehicle.get_transform() 
veh_loc = veh_trans.location
veh_rot = veh_trans.rotation
wpt = next_wp
wpt_loc = wpt.transform.location
wpt_rot = wpt.transform.rotation

print("veh loc: ", veh_loc)
print("wpt loc: ", wpt_loc)
print("veh rot: ", veh_rot)
print("wpt rot: ", wpt_rot)
print("road id: %d lane id: %d section id:%d "%(wpt.road_id, wpt.lane_id, wpt.section_id))
print("x dist: ", veh_loc.x - wpt_loc.x, "y dist: ", veh_loc.y - wpt_loc.y)
print("x-y dist: ", np.linalg.norm(np.array([veh_loc.x - wpt_loc.x, veh_loc.y - wpt_loc.y])))
print("rel ang: ", veh_rot.yaw - wpt_rot.yaw)

veh loc:  Location(x=189.929993, y=121.209999, z=0.215703)
wpt loc:  Location(x=189.708725, y=141.087311, z=0.000000)
veh rot:  Rotation(pitch=0.000000, yaw=89.999954, roll=0.000000)
wpt rot:  Rotation(pitch=360.000000, yaw=90.077835, roll=0.000000)
road id: 15 lane id: 1 section id:0 
x dist:  0.2212677001953125 y dist:  -19.87731170654297
x-y dist:  19.878543208047713
rel ang:  -0.077880859375


In [15]:

vehicle_transform = vehicle.get_transform()
distance_to_wp = wpt.transform.location.distance(vehicle_transform.location)
direction_difference = (vehicle_transform.rotation.yaw - wpt.transform.rotation.yaw) % 180
print(distance_to_wp)
print(direction_difference)

19.616851806640625
0.00492095947265625


In [17]:
vehicle_location  = vehicle.get_transform().location
waypoint_distance  = vehicle_location.distance(wpt.transform.location)
print(waypoint_distance)



19.616851806640625


In [18]:
current_waypoint = wpt
next_wp = current_waypoint.next(5.0)[0]
print(next_wp.transform.location)

Location(x=41.854179, y=237.590179, z=0.000000)


In [43]:
destination = next_wp.transform.location
agent.set_destination(destination)

In [44]:
while True:
    if agent.done():
        break
    vehicle.apply_control(agent.run_step())

KeyboardInterrupt: 

In [14]:
while True:
    if agent.done():
        next_wp = selected_wp.next(5.0)[0]
        destination = next_wp.transform.location
        agent.set_destination(destination)
        world.debug.draw_string(next_wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=255, b=0), life_time=2.0,
        persistent_lines=True)
        print("The target has been reached, setting next waypoint")
        selected_wp = next_wp
        waypoint_counter += 1

    if waypoint_counter > 100:
        print("We have reached the end of the waypoints")
        break
    vehicle.apply_control(agent.run_step())

The target has been reached, setting next waypoint


KeyboardInterrupt: 

In [16]:
# example how to use this for measuring where the car is in relation to where it should be

vehicle_transform = vehicle.get_transform()
distance_to_wp = selected_wp.transform.location.distance(vehicle_transform.location)
direction_difference = (vehicle_transform.rotation.yaw - selected_wp.transform.rotation.yaw) % 180
print('deviation from waypoint:',distance_to_wp,'metres')
print('angle discrepancy:',direction_difference,'degrees')

print("rel ang: ", veh_rot.yaw - wpt_rot.yaw)

deviation from waypoint: 214.70864868164062 metres
angle discrepancy: 90.00496673583984 degrees


NameError: name 'veh_rot' is not defined

In [11]:
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [45]:
vehicle_transform = vehicle.get_transform()
next_wp = selected_wp.next(20.0)[0]
distance_to_wp = selected_wp.transform.location.distance(vehicle_transform.location)
direction_difference = (vehicle_transform.rotation.yaw - selected_wp.transform.rotation.yaw) % 180
print(distance_to_wp)

0.5521860122680664
